In [1]:
import pandas as pd
from datetime import datetime
import ast
import gzip


In [2]:
with gzip.open('datasets/user_items_proc.csv.gz', 'rb') as f:
    user_items_proc = pd.read_csv(f, encoding='utf-8')
    
steam_games = pd.read_csv('./datasets/steam_games.csv', parse_dates=['release_date'])
user_review = pd.read_csv('./datasets/user_reviews.csv')

In [ ]:


diner_user = pd.merge(steam_games[['app_name','price']],user_items_proc[['user_id','items_count','item_name']],left_on='app_name',right_on='item_name',how='inner')
users_items = diner_user.groupby('user_id').agg(
    {
       'items_count':'first',
        'price':'sum'
    }
).reset_index()

recomendaciones_true = user_review.groupby('user_id')['recommend'].sum()
total_registros_por_usuario = user_review.groupby('user_id')['recommend'].count()
porcentaje_recomendaciones_true = (recomendaciones_true / total_registros_por_usuario) * 100
users_true_reviews = pd.DataFrame({
    'user_id': porcentaje_recomendaciones_true.index,
    'porcentaje_recomendaciones_true': porcentaje_recomendaciones_true.values
})
items_reviews_users = pd.merge(users_items,users_true_reviews,on='user_id',how='inner')
items_reviews_users.to_csv('./datasets/items_reviews_users.csv',index=False)

In [3]:
generos = steam_games.drop(columns=['app_name','release_date','specs','price','id','developer','Year','Accounting'])
gener_user = pd.merge(user_items_proc[['item_name','playtime_forever','user_id']],steam_games,left_on='item_name',right_on='app_name',how='inner')
resultados_por_genero = []
for genero in generos.columns:
    genero_data = gener_user[gener_user[genero] == 1]
    
    if not genero_data.empty:
        user_max = genero_data.groupby('user_id')['playtime_forever'].sum().idxmax()
        result_df = genero_data[genero_data['user_id'] == user_max].groupby('Year')['playtime_forever'].sum().reset_index()
        formatted_result = [{'Año': record['Year'], 'Horas': record['playtime_forever']} for _, record in result_df.iterrows()]
        resultados_por_genero.append({'Género': genero, 'Usuario': user_max, 'Año_Horas': formatted_result})
    else:
        resultados_por_genero.append({'Género': genero, 'Usuario': None, 'Año_Horas': []})

df_resultados = pd.DataFrame(resultados_por_genero)

df_resultados.dropna(inplace=True)
df_resultados.to_csv('./datasets/max_por_gen.csv',index=False)

In [25]:
def best_developer_year_func(año:str):
    func_4 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_4 = func_4[func_4['Year'] == float(año)]
    mejores_dev = func_4.groupby('developer')['recommend'].sum().reset_index().sort_values(by='recommend',ascending=False)
    if mejores_dev.empty:
        return 'No se enocntraron reviews para items que hayan salido ese año'
    else:
        puesto1 = mejores_dev.iloc[0][0]
        puesto2 = mejores_dev.iloc[1][0]
        puesto3 = mejores_dev.iloc[2][0]
        puestos = {"Puesto 1": puesto1, "Puesto 2": puesto2, "Puesto 3": puesto3}
        return puestos

In [26]:
best_developer_year_func(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Freejam',
 'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}

In [ ]:
user_review['posted'].iloc[0].split(',')[-1].strip(' .')

In [23]:
def developer_rec_func(desarrolladora:str):
    
    func_5 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
    func_5['developer'] = func_5['developer'].astype(str)
    func_5['developer'] = func_5['developer'].str.lower()
    desarrolladora = desarrolladora.lower()
    func_5 = func_5[func_5['developer'] == desarrolladora]
    if func_5.empty:
        return 'No se enocntraron reviews para items perteneceintes a esa desarroladora'
    else:
        true_value = func_5[func_5['sentiment_analysis']==2]['sentiment_analysis'].count()
        false_value = func_5[func_5['sentiment_analysis']==0]['sentiment_analysis'].count()
        return {desarrolladora:[f'Negative = {int(false_value)}',f'Positive = {int(true_value)}']}

In [24]:
developer_rec_func('Tripwire Interactive')

{'tripwire interactive': ['Negative = 109', 'Positive = 319']}

In [10]:
func_5 = pd.merge(user_review,steam_games,left_on='item_id',right_on='id',how='inner')
true_value = func_5[func_5['sentiment_analysis']==2]['sentiment_analysis'].count()
false_value = func_5[func_5['sentiment_analysis']==0]['sentiment_analysis'].count()

In [20]:
func_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50154 entries, 0 to 50153
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_id                    50154 non-null  object        
 1   posted                     50154 non-null  object        
 2   item_id                    50154 non-null  int64         
 3   recommend                  50154 non-null  bool          
 4   sentiment_analysis         50154 non-null  int64         
 5   app_name                   50154 non-null  object        
 6   release_date               49929 non-null  datetime64[ns]
 7   specs                      50154 non-null  object        
 8   price                      50154 non-null  float64       
 9   id                         50154 non-null  int64         
 10  developer                  50154 non-null  object        
 11  Accounting                 50154 non-null  int64         
 12  Acti